In [1]:
import xarray as xr
import numpy as np
import pandas as pd

def normal_to_east(x):
    "from -180~180 to 0~360, to eastern degrees"
    x[x<0] = x[x<0] + 360
    return x

def east_to_normal(x):
    "from 0~360 to -180~180"
    x[x>180] = x[x>180] - 360
    return x

def add_tierney_sst(data, lookup_data, column_name="SST"):
    lst = []
    latitude = data['Latitude'].copy()
    longitude = data['Longitude'].copy()
    longitude = normal_to_east(longitude)

    for i in range(len(longitude)):
        lat = latitude[i]
        lon = longitude[i]
        x = lookup_data.sel(lat=lat,
                         lon=lon,
                         method="nearest",
                         tolerance=1).values

        if np.isnan(x):
            x = lookup_data.sel(lat=lat,
                             lon=lon,
                             method="pad").values
        if np.isnan(x):
            x = lookup_data.sel(lat=lat,
                             lon=lon,
                             method="backfill").values
        lst.append(x)
    data[column_name] = np.array(lst)

def add_Had_data(data, lookup_data, column_name="SST"):
    lst = []
    latitude = data['Latitude'].copy()
    longitude = data['Longitude'].copy()

    for i in range(len(longitude)):
        lat = latitude[i]
        lon = longitude[i]
        x = lookup_data.sel(latitude=lat,
                         longitude=lon,
                         tolerance=1,
                         method="nearest").values
        lst.append(x)
    data[column_name] = np.array(lst)


def add_deg_sst(data, lookup_data, column_name="SST"):
    lst = []
    latitude = data['Lat'].to_numpy().copy()
    longitude = data['Lon'].to_numpy().copy()
    age = data['Age_kaBP'].to_numpy().copy()
    age = age.round() * 1000
    longitude = normal_to_east(longitude)

    for i in range(len(longitude)):
        lat = latitude[i]
        lon = longitude[i]
        time = age[i]

        x = lookup_data.sel(lat=lat, lon=lon, method="nearest").sel(age=time, method="nearest", tolerance=500).values
        print(lat, lon, time, x)

        if np.isnan(x):
            x = lookup_data.sel(lat=lat, lon=lon, method="pad").sel(age=time, method="nearest").values

        if np.isnan(x):
            x = lookup_data.sel(lat=lat, lon=lon, method="backfill").sel(age=time, method="nearest").values

        lst.append(x)
    data[column_name] = np.array(lst)

In [ ]:
# plot the optimal niche
# question: relative abundance is removing the effects of nutrient availbility

mod_paths = [
    "../model/LGM_8P7Z4F_gmd",
    "../model/muffin.CB.worlg4.BASESFeTDTL.FORAM.SPIN",
    "../model/pyyyyz.RpCO2_Rp13CO2.4deg.Albani.Foram.Transient",
]

foram = ["bn", "bs", "sn", "ss"]

fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(8, 8),
                        subplot_kw={'projection': ccrs.EckertIV()},
                        tight_layout=True)

counter = 0

for j in foram:
    for i in mod_paths:
        data = fm(i).select_foramtype(j).relative_abundance().isel(time=-1)

        # drop zeros and get 75th percentile
        data_nozero = data.filter(">", 0, overwrite_array=True).filter("<", 1, overwrite_array=True)
        upper_quartile = np.nanpercentile(data_nozero.array, 75)

        # filter data, but read again
        data = fm(i).select_foramtype(j).relative_abundance().isel(time=-1)
        filtered_data = data.filter(">", upper_quartile, overwrite_array=True)

        # plot
        filtered_data.plot_map(ax = axs.flat[counter], cmap="viridis")
        print(f"ploting {j} in {i}, with threshold in {upper_quartile}")
        counter += 1

#fig.savefig("../output/optimal_niche_distribution.jpg", dpi=400)

In [2]:
## Read in raw foram abundance data
core_list = pd.read_csv("../data/Strack_2022/CoreList_PlanktonicForaminifera.csv")
foram_abundance = pd.read_csv("../data/Strack_2022/FullDataTable_PF_harmonized.txt", delimiter="\t")
## merge site information
core_list = core_list[['Site', 'Lat', 'Lon']]
foram_abundance = pd.merge(core_list, foram_abundance, left_on='Site', right_on='Site')
## filter age
foram_abundance = foram_abundance[foram_abundance['Age_kaBP'] < 21]
foram_abundance = foram_abundance[foram_abundance['Age_kaBP'] >= 0]
# subset site information for get temperature
site_temp = foram_abundance[['Site', 'Lat', 'Lon', 'Age_kaBP']].drop_duplicates()

### read in raw sst data with pre-processing
deg_sst = xr.open_dataset("../data/deglaciation_climate/LGMR_SST_climo.nc", drop_variables=['lat', 'lon'])
## read in lat/lon
from netCDF4 import Dataset
data = Dataset("../data/deglaciation_climate/LGMR_SST_climo.nc")
sst_lon = data['lon'][0,:]
sst_lat = data['lat'][:,0]
data.close()

## resolve the issue of longitude
deg_sst['lat'] = sst_lat
deg_sst['lon'] = sst_lon
deg_sst = deg_sst.sortby('lon')
deg_sst = deg_sst['sst']

add_deg_sst(site_temp, deg_sst, "SST")
# add back to foram abundance data table
foram_abundance = pd.merge(site_temp, foram_abundance, on=['Site', 'Lat', 'Lon', 'Age_kaBP'])
foram_abundance.to_csv("../data/Strack_2022/deglacial_foram_with_sst.csv", index=False)

/var/folders/rq/vks10_qx2l9b08d_pygvwfzh0000gn/T/ipykernel_28595/25455440.py:18: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  sst_lon = data['lon'][0,:]
/var/folders/rq/vks10_qx2l9b08d_pygvwfzh0000gn/T/ipykernel_28595/25455440.py:19: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  sst_lat = data['lat'][:,0]


20.7492 341.4192 1000.0 22.06354331970215
20.7492 341.4192 2000.0 21.98687171936035
20.7492 341.4192 2000.0 21.98687171936035
20.7492 341.4192 3000.0 22.138256072998047
20.7492 341.4192 4000.0 22.173839569091797
20.7492 341.4192 4000.0 22.173839569091797
20.7492 341.4192 5000.0 22.310705184936523
20.7492 341.4192 5000.0 22.310705184936523
20.7492 341.4192 6000.0 22.395061492919922
20.7492 341.4192 6000.0 22.395061492919922
20.7492 341.4192 7000.0 22.42450714111328
20.7492 341.4192 8000.0 22.254064559936523
20.7492 341.4192 8000.0 22.254064559936523
20.7492 341.4192 9000.0 22.30426025390625
20.7492 341.4192 9000.0 22.30426025390625
20.7492 341.4192 10000.0 22.175460815429688
20.7492 341.4192 10000.0 22.175460815429688
20.7492 341.4192 11000.0 21.466894149780273
20.7492 341.4192 11000.0 21.466894149780273
20.7492 341.4192 12000.0 20.790042877197266
20.7492 341.4192 13000.0 20.729610443115234
20.7492 341.4192 13000.0 20.729610443115234
20.7492 341.4192 14000.0 20.566978454589844
20.7492 3

In [15]:
env_data = xr.open_dataset("../data/lgm_env/Tierney2020_DA_ocn_regrid.nc")

## longitude checked, regular but in eastern format
SST_LGM = env_data['SSTLGM']
SST_HOL = env_data['SSTLH']
SSS_LGM = env_data['SSSLGM']
SSS_HOL = env_data['SSSLH']

lgm_fg_a = pd.read_csv("~/foram_core/tidy/lgm_fg_a_tidy.csv")
lgm_fg_r = pd.read_csv("~/foram_core/tidy/lgm_fg_r_tidy.csv")
lgm_sp_a = pd.read_csv("~/foram_core/tidy/lgm_sp_a_tidy.csv")
lgm_sp_r = pd.read_csv("~/foram_core/tidy/lgm_sp_r_tidy.csv")

add_tierney_sst(lgm_fg_a, SST_LGM, "SST")
add_tierney_sst(lgm_fg_r, SST_LGM, "SST")
add_tierney_sst(lgm_sp_a, SST_LGM, "SST")
add_tierney_sst(lgm_sp_r, SST_LGM, "SST")

lgm_fg_a.to_csv("~/foram_core/tidy/lgm_fg_a_wsst.csv",index=False)
lgm_fg_r.to_csv("~/foram_core/tidy/lgm_fg_r_wsst.csv",index=False)
lgm_sp_a.to_csv("~/foram_core/tidy/lgm_sp_a_wsst.csv",index=False)
lgm_sp_r.to_csv("~/foram_core/tidy/lgm_sp_r_wsst.csv",index=False)

## ------------------------------

pi_fg_a = pd.read_csv("~/foram_core/tidy/forcens_fg_a_tidy.csv")
pi_fg_r = pd.read_csv("~/foram_core/tidy/forcens_fg_r_tidy.csv")
pi_sp_a = pd.read_csv("~/foram_core/tidy/forcens_sp_a_tidy.csv")
pi_sp_r = pd.read_csv("~/foram_core/tidy/forcens_sp_r_tidy.csv")

add_tierney_sst(pi_fg_a, SST_HOL, "SST")
add_tierney_sst(pi_fg_r, SST_HOL, "SST")
add_tierney_sst(pi_sp_a, SST_HOL, "SST")
add_tierney_sst(pi_sp_r, SST_HOL, "SST")

pi_fg_a.to_csv("~/foram_core/tidy/forcens_fg_a_wsst.csv",index=False)
pi_fg_r.to_csv("~/foram_core/tidy/forcens_fg_r_wsst.csv",index=False)
pi_sp_a.to_csv("~/foram_core/tidy/forcens_sp_a_wsst.csv",index=False)
pi_sp_r.to_csv("~/foram_core/tidy/forcens_sp_r_wsst.csv",index=False)

# forcens = pd.read_csv("../data/modern_foram/raw/forcens_raw_count.csv")
# forcens = forcens.dropna(subset=['Longitude', 'Latitude'])
# forcens = forcens.reset_index(drop=True)
# add_tierney_sst(forcens, SST_HOL, "SST")

In [28]:
jonkers = pd.read_csv("../data/Jonkers_2019/shell_flux_data.csv")
#jonkers['open'][1]
jonkers['open'] = pd.to_datetime(jonkers['open'])
jonkers['close'] = pd.to_datetime(jonkers['close'])
jonkers

,Unnamed: 0,me,short_me,lat,lon,trap_depth_m,ocean,upwelling,reference,data_source,...,H_pelagica,T_fleisheri,T_iota,G_uvula,S_dehiscens,C_nitida,G_adamsi,H_digitata,G_conglomerata,B_pumilio
0,1,Station Aleutian Basin,ABP,53.05,-177.00,3198,PAC,N,"Asahi, H., and Takahashi, K.: A 9-year time-se...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Station Aleutian Basin,ABP,53.05,-177.00,3198,PAC,N,"Asahi, H., and Takahashi, K.: A 9-year time-se...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Station Aleutian Basin,ABP,53.05,-177.00,3198,PAC,N,"Asahi, H., and Takahashi, K.: A 9-year time-se...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Station Aleutian Basin,ABP,53.05,-177.00,3198,PAC,N,"Asahi, H., and Takahashi, K.: A 9-year time-se...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Station Aleutian Basin,ABP,53.05,-177.00,3198,PAC,N,"Asahi, H., and Takahashi, K.: A 9-year time-se...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,1472,Western Arabian Sea Trap,WAS,16.32,60.47,3020,IND,Y,"Curry, W. B., Ostermann, D. R., Guptha, M. V. ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1472,1473,Western Arabian Sea Trap,WAS,16.32,60.47,3020,IND,Y,"Curry, W. B., Ostermann, D. R., Guptha, M. V. ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473,1474,Western Arabian Sea Trap,WAS,16.32,60.47,3020,IND,Y,"Curry, W. B., Ostermann, D. R., Guptha, M. V. ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1474,1475,Western Arabian Sea Trap,WAS,16.32,60.47,3020,IND,Y,"Curry, W. B., Ostermann, D. R., Guptha, M. V. ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def add_Had_data(data, lookup_data, column_name="SST"):
    lst = []
    latitude = data['lat'].copy()
    longitude = data['lon'].copy()
    open_time = data['open'].copy()
    close_time = data['close'].copy()

    for i in range(len(longitude)):
        lat = latitude[i]
        lon = longitude[i]
        print(lat, lon)
        x = lookup_data.sel(time=slice(open_time[i], close_time[i])).\
            sel(latitude=lat, longitude=lon, method='nearest', tolerance=1).mean(skipna=True).values
        lst.append(x)
    data[column_name] = np.array(lst)

Had_data = xr.open_dataset("../data/modern_env/HadISST_sst.nc")
Had_SST = Had_data['sst']

# mask sea-ice-covered gridboxes (i.e., -1000)
Had_SST = xr.where(Had_SST==-1000, np.nan, Had_SST)

# mask Land squares are set to -32768
Had_SST = xr.where(Had_SST==-32768, np.nan, Had_SST)

## slice pre-industrial mean
Had_SST_PI = Had_SST.sel(time=slice('1870-01-01', '1900-12-31')).mean(dim='time')
## export to netcdf
Had_SST_PI.to_netcdf('../data/modern_env/HadISST_PI.nc')

add_Had_data(jonkers, Had_SST, "SST")
jonkers.to_csv('../data/Jonkers_2019/shell_flux_data_wsst.csv')

53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0
53.05 -177.0